In [1]:
from astropy.io import ascii
import glob
import matplotlib.pyplot as plt
import numpy as np
from scipy import asarray as ar, exp
from scipy.optimize import curve_fit, least_squares
import re
%matplotlib inline

snls_colors = {"sdssg":"darkgreen", "sdssi":"darkblue", "sdssr":"darkred", "sdssz":"purple"}
base_colors = {"sdssg":"green", "sdssi":"blue", "sdssr":"red", "sdssz":"magenta"}

import astropy.coordinates as coord
import astropy.units as u
import Utils.DiaSourceTools as DSTools

import multiprocessing

import sncosmo

In [2]:
def read_lc_files(obj):
    table = ascii.read(obj)
    return table

In [3]:
f = open('/renoir_data_02/jpreyes/lsst_data/sn_control/J_A+A_523_A7_table9.dat.txt','r')
data_elems = f.read()
#elems = re.findall('^(.*?D3.*?)\\|(.*?)\\|', data_elems, re.MULTILINE)
elems = re.findall('^(05D3.*?)\\|(.*?)\\|(.*?)\\|(.*?)\\|(.*?)\\|(.*?)\\|(.*?)\\|', data_elems, re.MULTILINE)
f.close()

f = open('/renoir_data_02/jpreyes/lsst_data/sn_control/J_A+A_523_A7_table10.dat.txt','r')
data = f.read()
f.close()

In [4]:
snls_array = []
for sn in elems:

        c = coord.SkyCoord(sn[1], unit=(u.hourangle, u.deg))

        m = re.findall('^'+str(sn[0])+'\\|(r|g|z|i)\\|(.*?)\\|(.*?)\\|(.*?)$', data, re.MULTILINE)

        snls_lc = DSTools.build_light_curve_from_snls_file_2(m, c, sn[0], sn[-1])

        if len(m)>0:
            #print sn[0], c.ra.deg, c.dec.deg

            #plt.errorbar(snls_lc['mjd'], snls_lc['flux'], yerr=snls_lc['flux_error'], fmt='.', color='blue')
            #show_multi_filter_light_curve(snls_lc, snls_colors)
            plt.show()
            snls_array.append(snls_lc)

print("total of sn:" + str(len(snls_array)))

total of sn:33


In [5]:
def match_with_lc(slc):
    matches = []
    for lc in snls_array:
        #print "light curve"
        val = {"ra":lc[0]["ra"], "dec":lc[0]["dec"]}
        #print np.rad2deg(lc[0]["ra"]), np.rad2deg(lc[0]["dec"])
        

        comp = {"ra":np.mean(slc["ra"]), "dec":np.mean(slc["dec"])}
        if DSTools.source_distance(val, comp)<1:
            #print "match"
            matches.append((lc,slc))


    #print len(matches)
    return matches


In [6]:


def get_total_matches(sigma):
    lienObj = glob.glob("/datadec/cppm/jpreyes/CFHT_Production/light_curves_corrected_"+str(float(sigma))+"sigma/*.dat")



    try:
        pool = multiprocessing.Pool()
        res =  pool.map(read_lc_files, lienObj)
    finally:
        pool.close()

    s_candidates = res
    print len(s_candidates)

    try:
        pool = multiprocessing.Pool()
        match_res =  pool.map(match_with_lc, s_candidates)
    finally:
        pool.close()

    res_final = []
    for r in match_res:
        res_final +=r

    count = 0
    ids = []
    for control, match in res_final:
        if control[0]["id"] not in ids:
            count+=1
            ids.append(control[0]["id"])

    print count
    
    return s_candidates, res_final

In [7]:
def count_without_duplicates(res_final):
    count = 0
    ids = []
    for control, match in res_final:
        if control[0]["id"] not in ids:
            count+=1
            ids.append(control[0]["id"])

    return count


In [8]:
c, rf = get_total_matches(0)

67806
28


In [9]:
len(c)

67806

In [16]:
def get_total_matches_positives(c):

    selected_candidates = []



    for candidate in c:
        positives = len(candidate[candidate["classification"]==6])+len(candidate[candidate["classification"]==7])
        total = len(candidate)
        if float(positives)/float(total) > 0.5:
            selected_candidates.append(candidate)
    print len(selected_candidates)

    try:
        pool = multiprocessing.Pool()
        match_res =  pool.map(match_with_lc, selected_candidates)
    finally:
        pool.close()

    ms = []
    for r in match_res:
        ms +=r
    len(ms)
    print(count_without_duplicates(ms))

In [19]:
c, rf = get_total_matches(0)
get_total_matches_positives(c)

67806
28
32063
22


In [18]:
c, rf = get_total_matches(5)
get_total_matches_positives(c)
c, rf = get_total_matches(10)
get_total_matches_positives(c)
c, rf = get_total_matches(15)
get_total_matches_positives(c)
c, rf = get_total_matches(20)
get_total_matches_positives(c)
c, rf = get_total_matches(25)
get_total_matches_positives(c)

33188
24
38656
27
32931
24
38059
27
32649
24
37372
26
32244
23
36555
26
31651
23


In [65]:
get_total_matches("i",5)
get_total_matches("i",10)
get_total_matches("i",15)
get_total_matches("i",20)
get_total_matches("i",25)

57413
24
55989
24
54009
24
50791
24
45707
24


In [68]:
get_total_matches("g",5)
get_total_matches("g",10)
get_total_matches("g",15)
get_total_matches("g",20)
get_total_matches("g",25)

18197
10
17917
10
17605
10
17270
10
16789
10


In [ ]:
get_total_matches("r",0)
get_total_matches("i",0)
get_total_matches("g",0)



46382


In [77]:
A = {"1": {"1":10, "2":20}, "2": {"1":50, "2":60}}
B = {"1": {"1":100, "2":200}, "2": {"1":500, "2":600}}
C = {"1": {"1":1000, "2":2000}, "2": {"1":5000, "2":6000}}
tot = {"A":A, "B":B, "C":C}

In [78]:
tot["A"]["2"]["2"]

60

In [87]:
import pickle

filters = ["r","i","g"]

sigma_dicts = {}

for f in filters:
        sigma_dicts[f] =  pickle.load( open( "/datadec/cppm/jpreyes/CFHT_Production/sigma_dictionary_"+f+".pickle", "rb" ) )


In [88]:
sigma_dicts["g"]["8,12"]["2005-01-16"]

KeyError: '2005-01-16'

In [89]:
sigma_dicts["g"]

{'0,10': {},
 '0,11': {},
 '0,12': {},
 '0,13': {},
 '0,14': {},
 '0,15': {},
 '0,16': {},
 '0,17': {},
 '0,18': {},
 '0,19': {},
 '0,20': {},
 '0,21': {},
 '0,22': {},
 '0,23': {},
 '0,24': {},
 '0,25': {},
 '0,5': {},
 '0,6': {},
 '0,7': {},
 '0,8': {},
 '0,9': {},
 '1,10': {},
 '1,11': {},
 '1,12': {},
 '1,13': {},
 '1,14': {},
 '1,15': {},
 '1,16': {},
 '1,17': {},
 '1,18': {},
 '1,19': {},
 '1,20': {},
 '1,21': {},
 '1,22': {},
 '1,23': {},
 '1,24': {},
 '1,25': {},
 '1,5': {},
 '1,6': {},
 '1,7': {},
 '1,8': {},
 '1,9': {},
 '10,10': {'2005-01-20': 0.06465906445522786,
  '2005-02-10': 0.05780222441781209,
  '2005-03-17': 0.06192993037409178,
  '2005-04-02': 0.13223524101489964,
  '2005-04-03': 0.10793722061912787,
  '2005-04-06': 0.057697002514627886,
  '2005-04-09': 0.06320391503625512,
  '2005-04-13': 0.0724434689430056,
  '2005-05-05': 0.06355024950395703,
  '2005-05-10': 0.07026002389888289,
  '2005-05-12': 0.07420306126794679,
  '2005-05-15': 0.10260034036567292,
  '2005-05-